## Question 1

#### <ins>Equation A</ins>
The equation $x+y-z=0$ is valid in a linear program because it is a linear equation.

#### <ins>Equation B</ins>
The equation $x \le \frac{100}{y}$ is not valid in a linear program because it is a non-linear function.


#### <ins>Equation C</ins>
The equation $3x+2y \le \sqrt{5}$ is valid in a linear program because it is a linear function.


#### <ins>Equation D</ins>
The equation $\sqrt{5}x+2y = 50$ is valid in a linear program because it is a linear function.


#### <ins>Equation E</ins>
The equation $\sqrt{5x}+10y = 100$ is not valid in a linear program because it is a non-linear function as a result of $\sqrt{5x}$.


#### <ins>Equation F</ins>
The equation $x^{2}+y^{2} \ge 45$ is not valid in a linear program because it contains quadratic terms.


## Question 2

2.	Formulate and solve the “Two Products – Three Machine” problem from Class #1 as a linear program.  Also, how would you set up this model so you could scale it?  That is, how would you structure it if you had a lot of machines and a lot of products?

## Question 3

3.	Your non-vegetarian friend wants to determine how many units of different kinds of meat he should eat to get the right amounts of vitamins A, C, B1, and B2 over the week.  He also wants to minimize his cost.  What should his diet be?  How would you modify the linear program to make it “better?”  You can find the cost for each unit of food and the percent of daily requirements for each vitamin type in the spreadsheet. 

## Question 4

4.	Solve the gas blending problem introduced in class.  That is, determine how much of each type of gas you should buy and what you should use it for.  Make sure you can explain the full model and especially the blending constraints.  

## Question 5

5.	Solve the nurse scheduling problem introduced in class.  You want to know the minimum number of nurses needed to cover the 24-hour period.  You can assume that the next day is the same as this one.

## Question 6

In [47]:
import gurobipy as gp

# Create a new model
model = gp.Model("cutting_stock")

# Orders: [Width, Number of Rolls]
orders = [(5, 150), (7, 200), (9, 300)]

# Generate all combinations to cut 20-foot rolls into widths of 5, 7, and 9 feet
possible_cuts = []
for cut_5 in range(5):
    for cut_7 in range(3):
        for cut_9 in range(3):
            if cut_5 * 5 + cut_7 * 7 + cut_9 * 9 <= 20:
                possible_cuts.append((cut_5, cut_7, cut_9))

# Decision Variables
x = {}
for i, cut in enumerate(possible_cuts):
    x[i] = model.addVar(vtype=gp.GRB.INTEGER, name=f"x_{cut}")

# Objective Function: Minimize the total number of 20-foot rolls
model.setObjective(sum(x[i] for i in x), gp.GRB.MINIMIZE)

# Constraints: Meet the total demand for each order
for j, (width, demand) in enumerate(orders):
    model.addConstr(
        sum(x[i] * possible_cuts[i][j] for i in range(len(possible_cuts))) >= demand, 
        name=f"demand_{j}"
    )

# Solve the model
model.optimize()

# Output Results
print("\nOptimal Solution:")
for v in model.getVars():
    if v.x > 0:  # Only show cuts that are actually used
        print(f"{v.varName}: {v.x}")
print(f"Minimum number of 20-foot rolls needed: {model.objVal}")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i9-10850K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 3 rows, 15 columns and 20 nonzeros
Model fingerprint: 0xab337bf2
Variable types: 0 continuous, 15 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 3e+02]
Found heuristic solution: objective 500.0000000
Presolve removed 0 rows and 9 columns
Presolve time: 0.00s
Presolved: 3 rows, 6 columns, 10 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)

Root relaxation: objective 2.625000e+02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  262.50000    0    1  500

In [58]:
# Initialize total leftover length to zero
total_leftover_length = 0

# Sum up the total leftover length based on the optimal solution
for i, v in enumerate(model.getVars()):
    if v.x > 0:
        cut_5, cut_7, cut_9 = possible_cuts[i]
        used_length = cut_5 * 5 + cut_7 * 7 + cut_9 * 9
        leftover_length = 20 - used_length  # leftover material for one roll of this cut
        total_leftover_length += int(v.x) * leftover_length  # sum it up for all rolls of this cut

# Print total leftover length
print(f"Total leftover length: {total_leftover_length}")

Total leftover length: 401


In [60]:
(263 * 20) - (5 * 150 + 7 * 200 + 9 * 300)

410